In [ ]:
# pip install sentence_transformers
# pip install langchain
# pip install pypdf
# pip install typing-extensions
# pip install chromadb
# pip install unstructured

In [ ]:
from langchain.llms import LlamaCpp

MODEL_PATH = "../models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
MODEL_PATH = "/home/flash/models/dolphin-2.5-mixtral-8x7b.Q4_K_M.gguf"
MODEL_HOME = "../models"
MODEL_PATH = f"{MODEL_HOME}/amethyst-13b-mistral.Q4_K_M.gguf"
# 38min
# 40s
llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_ctx=1024 * 4,
    n_gpu_layers=43, #43
    n_threads=15,
    n_batch=512,
    f16_kv=True,
    #callback_manager=callback_manager,
    verbose=False,
    temperature=0.1,
    max_tokens=2048
)

In [ ]:
def ask(user_input):
   prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{user_prompt}

### Response:
"""
   prompt = prompt_template.format(user_prompt=user_input)
   resp = llm(prompt,  
      max_tokens=500,
      temperature=0.7,
      top_p=0.1,
      repeat_penalty=1.1)
   return resp

In [ ]:
answer = ask("What is your name?")
answer

In [ ]:
answer = ask("Generate similar questions for \"What is payout of bet options in baccarat?\"")
answer

In [ ]:
question = "What is payout of bet options in baccarat?"
answer = ask(f"Generate multiple search queries related to: {question} \n OUTPUT (4 queries):")
answer

In [ ]:
def convert_generate_queries(answer):
   generate_queries = answer.split("\n")

# Load Embedding

In [ ]:
from py_standard.langchain_lit import split_documents, convert_docs_to_splits, LlmEmbedding, load_markdown_documents

EMB_MODEL = "bge-base-en"
#EMB_MODEL = "bge-large-zh-v1.5"

llm_embedd = LlmEmbedding(f"../models/{EMB_MODEL}")

In [ ]:
from py_standard.langchain_lit import LlmEmbedding, load_all_documents, create_chroma_vectorstore
from py_standard.pdf_utils import load_pdf_documents_from_directory
from langchain.vectorstores import Chroma

In [ ]:
docs = load_all_documents('./documents')
len(docs)

In [ ]:
vector_store = create_chroma_vectorstore(llm_embedd.embedding, "sample")
vector_store.add_documents(docs)

In [ ]:
def search_docs(user_query):
   global vector_store
   docs = vector_store.similarity_search_with_score(user_query, k=5)
   return docs

In [ ]:
sub_docs = search_docs("What is your name?")
sub_docs

In [ ]:
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage import LocalFileStore

def create_persist_docstore(persist_directory="./results/docstore/"):
   fs = LocalFileStore(persist_directory)
   store = create_kv_docstore(fs)
   return store

In [76]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.prompts import PromptTemplate
from IPython.display import display, Markdown, Latex


def create_parent_document_retriever(vector_store, docs):
   # store for parent
   # store = InMemoryStore() 
   store = create_persist_docstore()
   parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
   child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
   big_chunks_retriever = ParentDocumentRetriever(
      vectorstore=vector_store,
      docstore=store,
      parent_splitter=parent_splitter,
      child_splitter=child_splitter,
   )
   big_chunks_retriever.add_documents(docs, ids=None)
   return big_chunks_retriever

def create_qa_chain1(llm, retriever):
   qa_chain = RetrievalQA.from_chain_type(llm,
                                          chain_type='stuff',
                                          retriever=retriever)
   return qa_chain

def ask_qa_chain1(qa_chain, user_question):
   resp = qa_chain(user_question)
   answer = resp['result']
   return answer

def create_qa_chain2(llm, retriever):
   question_prompt = PromptTemplate.from_template(
      """You are QA Bot. If you don't know the answer, just say that you don't know, don't try to make up an answer.""")
   question_prompt = PromptTemplate.from_template(
      """You will use the provided context to answer user questions.
Read the given context before answering questions and think step by step.
If you can not answer a user question based on the provided context, respond with \"I don't know.\".
Do not use any other information for answering user.""")
   qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      retriever=retriever,
      condense_question_prompt=question_prompt,
      return_source_documents=True,
      verbose=False
   )
   return qa_chain

def ask_qa2(user_question):
   global qa_chain2
   history = []
   result = qa_chain2({"question": user_question, "chat_history": history})
   source_documents = result['source_documents']
   print(f"{source_documents=}")
   # source = result['metadata']['source']
   history = [(user_question, result["answer"])]
   return result["answer"]

In [ ]:
vector_store = create_vectorstore("./results/full_db")
full_retriever = create_parent_document_retriever(vector_store, docs)

In [ ]:
qa_chain1 = create_qa_chain1(llm, full_retriever)

In [69]:
answer = ask_qa_chain1(qa_chain1, "What is your name?")
answer

" I don't know the names of the other characters in this context."

In [70]:
qa_chain2 = create_qa_chain2(llm, full_retriever)

In [ ]:
# short document
sub_docs = vector_store.similarity_search_with_score("what is your name?", k=5)
sub_docs

In [ ]:
# full document
sub_docs = full_retriever.get_relevant_documents("What is your name?")
sub_docs

In [ ]:
sub_docs = full_retriever.get_relevant_documents("What functionality was released on 8/12/2023?")
sub_docs

In [ ]:
sub_docs = full_retriever.get_relevant_documents("What is Five-Count Baccarat?")
sub_docs

In [77]:
answer = ask_qa2("What is your name?")
answer

source_documents=[Document(page_content='What is your name?\nMy name is Allen.', metadata={'source': 'documents/t1.md', 'doc_id': 9}), Document(page_content='What is your name?\nMy name is Astro. Flash created me.\n\nOn August 12, 2023, the Eureka team released this feature. In Leo, specific handling was implemented to conceal transactions originating from specific hardcoded salary accounts across two pages (Txn and Sal Txn - Approval Sal Txn).\n\nWhat is your name?\nMy name is Jack.', metadata={'source': 'documents/t2.md', 'doc_id': 10}), Document(page_content='What is your name?\nMy name is Astro. Flash created me.\n\nOn August 12, 2023, the Eureka team released this feature. In Leo, specific handling was implemented to conceal transactions originating from specific hardcoded salary accounts across two pages (Txn and Sal Txn - Approval Sal Txn).', metadata={'source': 'documents/test.md', 'doc_id': 11}), Document(page_content='On August 12, 2023, the Eureka team released this feature.

" I don't know."

In [ ]:
answer = ask_qa2("What functionality was released on 8/12/2023?")
answer

In [ ]:
answer = ask_qa2("What is Five-Count Baccarat?")
answer

In [62]:
answer = ask_qa2("How do conduct a code review?")
answer

' To conduct a code review, follow these steps:\n\n1. Understand the requirements: Ensure that you have a clear understanding of the problem being solved by the code and its intended audience. Confirm security requirements and permission controls if applicable.\n2. Review code quality and readability: Examine the code for adherence to coding standards, best practices, and readability. Look for areas where the code can be simplified or optimized without compromising functionality.\n3. Assess testing and validation: Evaluate the current testing status, achieved test coverage, and adequacy of edge case and boundary condition testing. Understand how performance tests and metrics are used to ensure the code meets its requirements.\n4. Review code presentation: Walk through each code change section with the developer, explaining the purpose of each modification. Ensure that the code is modular and well-organized for reusability.\n5. Documentation review: Verify that all necessary documentati

In [67]:
answer = ask_qa2("List the items the reviewer needs to address in a code review.")
answer

result={'question': 'List the items the reviewer needs to address in a code review.', 'chat_history': [], 'answer': " The reviewer should address the following items during a code review:\n\n1. Understanding requirements: Ensure that the solution aligns with the defined requirements and that the target audience's specific needs are met. Confirm security requirements and permission control.\n2. Code quality and readability: Assess if the code is readable, understandable without excessive comments, follows coding standards and best practices, and can be simplified or optimized in any areas.\n3. Testing and validation: Evaluate the current testing status, achieved test coverage, edge case and boundary condition coverage, and performance tests and metrics used.\n4. Code presentation: Review each code change section, explaining their purpose, and ensure that the code is modular and well-organized for reusability.\n5. Documentation: Verify that all necessary documentation and requirements ar

" The reviewer should address the following items during a code review:\n\n1. Understanding requirements: Ensure that the solution aligns with the defined requirements and that the target audience's specific needs are met. Confirm security requirements and permission control.\n2. Code quality and readability: Assess if the code is readable, understandable without excessive comments, follows coding standards and best practices, and can be simplified or optimized in any areas.\n3. Testing and validation: Evaluate the current testing status, achieved test coverage, edge case and boundary condition coverage, and performance tests and metrics used.\n4. Code presentation: Review each code change section, explaining their purpose, and ensure that the code is modular and well-organized for reusability.\n5. Documentation: Verify that all necessary documentation and requirements are provided to the reviewer."

In [ ]:
# pip install rank_bm25
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 4

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from llama_index import StorageContext, ServiceContext, VectorStoreIndex, SimpleDirectoryReader

def load_llama_index_documents(docs_directory):
   reader = SimpleDirectoryReader(input_dir=docs_directory,
                                  required_exts=['.md', '.txt', '.pdf'],
                                  recursive=False)
   return reader.load_data()

# very slow
def create_persist_bm25_retriever(vector_store, documents):
   storage_context = StorageContext.from_defaults(vector_store=vector_store)
   service_context = ServiceContext.from_defaults(llm=llm, 
                                                  embed_model=llm_embedd.embedding, 
                                                  chunk_size=500,
                                                  chunk_overlap=50)
   
   index = VectorStoreIndex.from_documents(
      documents, 
      storage_context=storage_context, 
      service_context=service_context
   )
   
   # vector_retriever = index.as_retriever(similarity_top_k=5)
   bm25_retriever = BM25Retriever.from_defaults(
      docstore=index.docstore,
      similarity_top_k=5
   )
   return bm25_retriever

In [ ]:
bm25_docs = load_llama_index_documents("/home/flash/documents/")

In [ ]:
bm25_vector_store = create_vectorstore("/results/bm25")

In [ ]:
bm25_retriever = create_persist_bm25_retriever(bm25_vector_store, bm25_docs)

In [ ]:
bm25_retriever.get_relevant_documents("what is your name?")

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, full_retriever],
                                       weights=[0.4, 0.6])

In [ ]:
qa_chain3 = create_qa_chain1(llm, ensemble_retriever)

In [ ]:
answer = ask_qa_chain1(qa_chain3, "What is your name?")
answer

In [ ]:
qa_chain2 = create_qa_chain2(llm, ensemble_retriever)

In [ ]:
answer = ask_qa2("What is your name?")
answer

In [ ]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor,
                                                       base_retriever=ensemble_retriever)

# use this

In [ ]:
# compressor prompt
compressor.llm_chain.prompt

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents("What is your name?")
compressed_docs

In [ ]:
import langchain
langchain.debug = False

In [ ]:
compressor_qa_chain = create_qa_chain1(llm, compression_retriever)
answer = ask_qa_chain1(compressor_qa_chain, "How do you conduct a code review?")
answer

In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
_filter.llm_chain.prompt

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [ ]:
filter_compression_retriever = ContextualCompressionRetriever(base_compressor=_filter, base_retriever=compression_retriever)

sub_docs = filter_compression_retriever.get_relevant_documents("What is your name?")
pretty_print_docs(sub_docs)

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

llm_embedd = LlmEmbedding(f"../models/{EMB_MODEL}")

embeddings_filter = EmbeddingsFilter(embeddings=llm_embedd.embedding, similarity_threshold=0.76)
embedd_compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, 
                                                              base_retriever=filter_compression_retriever)


In [ ]:
sub_docs = embedd_compression_retriever.get_relevant_documents("What is your name?")
len(sub_docs)

In [ ]:
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder

embeddings = HypotheticalDocumentEmbedder.from_llm(llm,
                                                   llm_embedd.embedding,
                                                   prompt_key="web_search"
                                                   )

In [ ]:
result = embeddings.embed_query("What is your name?")
result

In [ ]:
prompt_template = """Please answer the user's question as related to Large Language Model
Question: {question}
Answer:"""

prompt = PromptTemplate(input_variables=["question"], template=prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

import langchain
langchain.debug = True

# 建立假設性的答案
embeddings = HypotheticalDocumentEmbedder(
    llm_chain=llm_chain,
    base_embeddings=llm_embedd.embedding
)

In [ ]:
result = embeddings.embed_query("What is your name?")

In [ ]:
docsearch = Chroma.from_documents(docs, embeddings)
query = "What is your name?"
docs = docsearch.similarity_search(query)
docs

####